# Cooling Tower Design <br> Harikrishnan R N , 18CHE147 <br> Vyankatesh Puri , 18CHE146

## Importing Necessary Modules

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import style
from scipy.optimize import fsolve, minimize
from scipy.integrate import trapz
import pandas as pd
from PIL import Image
import openpyxl
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 



%config InlineBackend.figure_format = 'retina'

style.use("classic")

%matplotlib inline

## Defining the Process Parameters and Constants

In [2]:
from openpyxl import load_workbook
workbook = load_workbook( filename = "Cooling_Tower.xlsx")
sheet = workbook['Design_Specs']

Td_air_in = sheet['D3'].value #31 # degCelcius, Dry bulb Temperature of inlet Air
Tw_air_in = sheet['D4'].value #22 # degCelcius, Wet bulb Temperature of Inlet Air
T_water_out = sheet['D5'].value #30 #degCelcius, Temperature of water in Outlet
T_water_in = sheet['D6'].value #45 #degCelcius, Temperature of water in Inlet
Lwater = sheet['D7'].value #6000 # kg/m^2hr, Mass flowrate of water
Ratio = sheet['D8'].value #1.4   # Ratio between Air flowrate and Minimum needed ar flowrate
cwl = sheet['D9'].value #4.178  # kJ/Kg.K , Specific heat of water
kya = sheet['D10'].value #6000    # kg/m^3.hr.delY', individual gas phase mass transfer coefficient

FileNotFoundError: [Errno 2] No such file or directory: 'Cooling_Tower.xlsx'

In [ ]:
sheet['D10'].value   # Test run

## Below I have defined functions to compute the Vapour pressure, Humidity and Enthalpy at Saturation Conditions

In [ ]:
Pt = 760 # mmHg, Atmospheric Pressure
def VapourPressure(T):
    """ T in degree Celcius, P in mmHg"""
    A = 18.3036
    B = -3816.44
    C = -46.13
    lnP = A + B/(T+273.16+C)
    return np.exp(lnP)

def HumiditySat(T):
    """ T in degree Celcius, RH is percentage Relative Humidity"""
    VP = VapourPressure(T)
    Y =  (VP/(Pt - VP))*(18.02/28.97)
    return Y

def EnthalpySat(T):
    """ T in degree Celcius, H in kJ/mol"""
    Y = HumiditySat(T)
    H = 2500*Y + (1.005 + 1.88*Y)*T
    return H 
 
    

## Plotting The Saturation Enthalpy Curve

In [ ]:
T = np.linspace(T_water_out*0.8,T_water_in*1.2,100)
VP = VapourPressure(T)
Y = HumiditySat(T)
H = EnthalpySat(T)
plt.figure(figsize = (7,7))
plt.plot(T, H)
plt.xlabel('Temperature, degC', fontsize = 15)
plt.ylabel('Saturation Enthalpy, kJ/kg', fontsize = 15)
plt.title('Saturation Enthalpy Curve', fontsize = 15)
plt.grid('True')
plt.savefig('graph_1.png')

## We know  $  (T_{G} \; - \; T_{S}) \; = \; (Y'_{S} \; - \; Y')\frac{\lambda_{S}}{c_{H}} $ so we have $Y'_{S} \; , \; T_{S} \; and \; T_{G}.$ we need  $Y'$ <br> Below I have defined functions to compute Humidity, Enthalpy at non saturation conditions, but using Wet Bulb and Dry Bulb Temperature

In [ ]:
def Humidity(Tg, Ts):
    Ys = HumiditySat(Ts)
    def objective(Y):
        f = ((Tg-Ts)*((1.005 + 1.88*Y))/2500) - Ys + Y
        return f
    Y0 = 0.1*np.ones(Ys.size)
    Y = fsolve(objective, Y0)
    return Y

def Enthalpy(Y,T):
    H = 2500*Y + (1.005 + 1.88*Y)*T
    return H
Tg, Ts = Td_air_in, Tw_air_in

## Locating the Point Q (Enthalpy of water at outlet Conditions)

In [ ]:
T = np.linspace(T_water_out*0.8,T_water_in*1.2,100)
VP = VapourPressure(T)
Y = HumiditySat(T)
H = EnthalpySat(T)
plt.figure(figsize = (7,7))
plt.plot(T, H)
plt.scatter([30], [Enthalpy(Humidity(T_water_out,Ts), T_water_out)], marker = "^")
plt.xlabel('Temperature, degC', fontsize = 15)
plt.ylabel('Saturation Enthalpy, kJ/kg', fontsize = 15)
plt.title('Saturation Enthalpy Curve', fontsize = 15)
plt.grid('True')
plt.annotate('Q', xy = (T_water_out*0.95,Enthalpy(Humidity(T_water_out,Ts), T_water_out)*1.1), fontsize = 15);
plt.savefig('graph_2.png')

## Now we find the line tangent to the Saturation Curve and passes through point Q

In [ ]:
def FindFlowrate(T_out):
    Q_T = T_out
    Q_H = Enthalpy(Humidity(T_out,Ts), T_out)
    def slopediff(T):
        h = 0.001
        x1 = Q_T
        y1 = Q_H
        x2 = T
        y2 = EnthalpySat(T)
        slope1 = (y2-y1)/(x2-x1)
        slope2 = (EnthalpySat(T+h)-EnthalpySat(T-h))/(2*h)
        error = (slope1 - slope2)**2
        return error
        
    
    T0 = T_water_in
    temp = minimize(slopediff, T0)
    return temp
        
def FindSlope(T1,T2):
    x1 = T1
    y1 = Enthalpy(Humidity(T1,Ts), T1)
    x2 = T2
    y2 = EnthalpySat(T2)
    slope = (y2-y1)/(x2-x1)
    return slope
    
    

In [ ]:
temp = FindFlowrate(T_water_out).x
m = FindSlope(T_water_out, temp)
T1 = np.linspace(T_water_out,temp*1.3,100)
plt.figure(figsize = (7,7))
plt.plot(T1, EnthalpySat(T1))
plt.plot(T1, Enthalpy(Humidity(T_water_out,Ts), T_water_out)+ m*(T1-T1[0]))
plt.scatter([30], [Enthalpy(Humidity(T_water_out,Ts), T_water_out)], marker = "^")
plt.xlabel('Temperature, degC', fontsize = 15)
plt.ylabel('Saturation Enthalpy, kJ/kg', fontsize = 15)
plt.title('Saturation Enthalpy Curve', fontsize = 15)
plt.grid('True')
plt.annotate('Q', xy = (T_water_out,Enthalpy(Humidity(T_water_out,Ts), T_water_out)*1.1), fontsize = 15);
plt.annotate('Tangent Line', xy = (35,100), fontsize = 15)
plt.annotate('Saturation Enthalpy Curve', xy = (35,300), fontsize = 15);
plt.savefig('graph_3.png')

## This parallel line gives us the slope, which is equal to ratio of Liquid  to minimum gas Flowrates: $ Slope \; = \; \frac{Lc_{wL}}{G_{s,min}} $ <br> From this we get the minimum required gas flowrate:


In [ ]:
Gs_min = Lwater*cwl/m
Gair = Gs_min*Ratio # Dry basis

## Drawing the Operating Line :

In [ ]:
# New slope of operating line will be Ratio (1.4 this case)  times smaller than the slope of the tangent we drew earlier.
m2 = m/Ratio 
# operating line equation : 
T2 = np.linspace(T_water_out, T_water_in, 100)
Op = Enthalpy(Humidity(T_water_out,Ts), T_water_out)+ m2*(T2-T2[0])
plt.figure(figsize = (7,7))
plt.plot(T2, EnthalpySat(T2))
plt.plot(T2, Op)
plt.scatter([T_water_out], [Enthalpy(Humidity(T_water_out,Ts), T_water_out)], marker = "^", linewidth = 8, color = 'blue')
plt.scatter([T_water_in], Op[-1], marker = "^", linewidth = 8, color = 'red')
plt.xlabel('Temperature, degC', fontsize = 15)
plt.ylabel('Enthalpy, kJ/kg', fontsize = 15)
plt.title('Saturation Enthalpy and Operating Line', fontsize = 12)
plt.grid('True')
plt.annotate('Q', xy = (T_water_out,Enthalpy(Humidity(T_water_out,Ts), T_water_out)*1.1), fontsize = 15);
plt.annotate('P', xy = (T2[-1], Op[-1]*1.05), fontsize = 15)
plt.annotate('Operating Line', xy = (40,100), fontsize = 15)
plt.annotate('Saturation Enthalpy Curve ', xy = (36,200), fontsize = 15);
plt.savefig('graph_4.png')

## Water side heat transfer coefficient is given by the correlation <br> $ h_{L}\overline{a} \; = \; 0.059L^{0.51}G_{S}$

In [ ]:
hla = 0.059*np.power(Lwater, 0.51)*Gair # kcal/hr.m^3
hla = hla*4.1858 # kJ/hr.m^3

### Slope of Tie-Lines :

In [ ]:
mtie = -hla/kya

## We have to make tie lines that start from the operating line that meet the saturation curve at the points $ (T_{L,i} \; , \; H'_{i})$ <br> We will define the necessary functions needed below:

In [ ]:
def OpLine(T):
    if (T > T_water_in) or (T < T_water_out):
        raise ValueError('Please enter value of T within temperature range')
    else:
        H = Enthalpy(Humidity(T_water_out,Ts), T_water_out)+ m2*(T-T2[0])
    return H

def TieLine(T):
    x1,y1 = T, OpLine(T)
    def TieInt(X):
        x2, y2 = X[0], X[1]
        err1 = y2 - EnthalpySat(x2)
        err2 = y1 - y2 + mtie*(x2 - x1)
        error = err1,err2
        return error
    x20,y20 = x1*0.9, y1*1.1
    X = np.zeros(2); X[0] = x20; X[1] = y20
    sol = fsolve(TieInt, X)
    x2,y2 = sol[0], sol[1]
    return (x2,y2)
    
def DrawTie(T):
    x1,y1 = T, OpLine(T);
    x2,y2 = TieLine(T);
    #plt.figure(figsize = (7,7))
    plt.plot(T2, EnthalpySat(T2), 'orange');
    plt.plot(np.linspace(25,30,100), EnthalpySat(np.linspace(25,30,100)), 'orange');
    plt.plot(T2, Op);
    plt.plot([x1,x2], [y1,y2], '--k');
    
    
    

## A plot with all the Tie Lines

In [ ]:
TieDraw = np.vectorize(DrawTie)
T3 = np.linspace(T_water_out, T_water_in, 10, dtype = float)
plt.figure(figsize = (7,7));
TieDraw(T3);
plt.scatter([T_water_out], [Enthalpy(Humidity(T_water_out,Ts), T_water_out)], marker = "^", linewidth = 8, color = 'blue');
plt.scatter([T_water_in], Op[-1], marker = "^", linewidth = 8, color = 'red');
plt.xlabel('Temperature, degC', fontsize = 15);
plt.ylabel('Enthalpy, kJ/kg', fontsize = 15);
plt.title('Saturation Enthalpy Curve, Operating line and Tie Lines', fontsize = 12);
plt.grid('True');
plt.annotate('Q', xy = (T_water_out,Enthalpy(Humidity(T_water_out,Ts), T_water_out)*1.1), fontsize = 15);
plt.annotate('P', xy = (T2[-1], Op[-1]*1.05), fontsize = 15)
plt.annotate('Operating Line', xy = (40,120), fontsize = 15);
plt.annotate('Saturation Enthalpy Curve', xy = (36,200), fontsize = 15);
plt.savefig('graph_5.png')

## Now we need to find the values of $ H'_{i} \; - \; H'$ <br> After we obtain that, we need to numerically integrate $ \int_{a}^{b} \frac{1}{(H'_{i} \; - \; H')} $ <br> While doing graphically, we may be limited by number of points to consider, but here, we can take larger number of points, so we will consider a sufficiently large number of points to numerically integrate

In [ ]:
# Defining the function to Integrate
TieLine_vec = np.vectorize(TieLine)
x2, Hi = TieLine_vec(T2)
OpLine_vec = np.vectorize(OpLine)
H = OpLine_vec(T2)
func = 1/(Hi - H)
Integral = trapz(func)

## The integral we obtained above is equal to the Number of Gas-Phase Transfer Units $ N_{tG} $ <br> Further Calculations : <br> Height of a transfer unit ($H_{tG}$) = $\frac{G_{s}}{k_{y*}\overline{a}}$ <br> Total packed height thus needed, z = $ N_{tG}H_{tG}$

In [ ]:
NTU = Integral 
HTU = Gair/kya
z = NTU*HTU

# Final Design Parameters : 

In [ ]:
data = {}
data['Design Parameter'] = ['Inlet Water Temperature (degC)','Outlet Water Temperature (degC)', 
                            'Inlet Air Dry bulb Temperature (degC)', 'Inlet Air Wet bulb Temperature (degC)',
                            'Inlet Water Flowrate (kg/m^2hr)', 'Inlet Air Flowrate (kg/m^2hr)',
                                'NTU', 'HTU', 'Tower Height (m)' ]
data['Value']=[T_water_in ,T_water_out, Td_air_in, Tw_air_in, Lwater, round(Gair[0],1), round(NTU,2), round(HTU[0],3), round(z[0],3) ]
table = pd.DataFrame(data = data)
table

In [ ]:
for i in range(len(table)):
    index1 = 'F'+str(i+3)
    sheet[index1].value = table['Design Parameter'][i]
    index2 = 'G'+str(i+3)
    sheet[index2].value = table['Value'][i]
    

In [ ]:
img = openpyxl.drawing.image.Image('graph_1.png')
img.height = 440 # insert image height in pixels as float or int (e.g. 305.5)
img.width = 440  # insert image height in pixels as float or int (e.g. 305.5)
img.anchor = 'A13'
sheet.add_image(img)

img = openpyxl.drawing.image.Image('graph_2.png')
img.height = 440 # insert image height in pixels as float or int (e.g. 305.5)
img.width = 440  # insert image height in pixels as float or int (e.g. 305.5)
img.anchor = 'G13'
sheet.add_image(img)

img = openpyxl.drawing.image.Image('graph_3.png')
img.height = 440 # insert image height in pixels as float or int (e.g. 305.5)
img.width = 440  # insert image height in pixels as float or int (e.g. 305.5)
img.anchor = 'A36'
sheet.add_image(img)

img = openpyxl.drawing.image.Image('graph_4.png')
img.height = 440 # insert image height in pixels as float or int (e.g. 305.5)
img.width = 440  # insert image height in pixels as float or int (e.g. 305.5)
img.anchor = 'G36'
sheet.add_image(img)

img = openpyxl.drawing.image.Image('graph_5.png')
img.height = 440 # insert image height in pixels as float or int (e.g. 305.5)
img.width = 440  # insert image height in pixels as float or int (e.g. 305.5)
img.anchor = 'A57'
sheet.add_image(img)


#workbook.insert_image('A13','graph_1.png')

In [ ]:
workbook.save('Cooling_Tower.xlsx')